#### LeetCode - SQL - #182

Refer [182. Duplicate Emails](https://leetcode.com/problems/duplicate-emails/description/)

Write a solution to report all the duplicate emails. Note that it's guaranteed that the email field is not NULL.

Return the result table in  **any order**.

In [0]:
data = [[1, 'a@b.com'], [2, 'c@d.com'], [3, 'a@b.com']]
columns = ['id', 'email']
schema = {'id':'Int64', 'email':'object'}

In [0]:
import pandas

emails_pandas_dataframe = pandas.DataFrame(data=data, columns=columns).astype(schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
emails_dataframe = spark_context.createDataFrame(emails_pandas_dataframe)
emails_dataframe.printSchema()
emails_dataframe.show(5)

root
 |-- id: long (nullable = true)
 |-- email: string (nullable = true)

+---+-------+
| id|  email|
+---+-------+
|  1|a@b.com|
|  2|c@d.com|
|  3|a@b.com|
+---+-------+



In [0]:
from pyspark.sql.functions import col, count

#1
emails_dataframe_grouped_1 = emails_dataframe.groupBy(emails_dataframe.email).count()
emails_dataframe_grouped_1 \
    .filter(col("count") > 1) \
    .select(emails_dataframe_grouped_1.email.alias("Email")) \
    .show()

#2
emails_dataframe_grouped_2 = emails_dataframe.groupBy(emails_dataframe.email).agg(
    count(emails_dataframe.id).alias("email_instance")
)
emails_dataframe_grouped_2 \
    .filter(emails_dataframe_grouped_2.email_instance > 1) \
    .select(emails_dataframe_grouped_1.email.alias("Email")) \
    .show()

+-------+
|  Email|
+-------+
|a@b.com|
+-------+

+-------+
|  Email|
+-------+
|a@b.com|
+-------+



#### Using Window Functions

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_spec = Window.partitionBy(emails_dataframe.email).orderBy(emails_dataframe.id)
emails_dataframe = emails_dataframe.withColumn("email_instance", row_number().over(window_spec))
emails_dataframe.show(5)

emails_dataframe \
    .filter(emails_dataframe.email_instance > 1) \
    .select(emails_dataframe.email.alias("Email")) \
    .distinct() \
    .show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4004504151319970>, line 10
      3 emails_dataframe_grouped_1 = emails_dataframe.groupBy(emails_dataframe.email).count()
      4 emails_dataframe_grouped_1 \
      5     .filter(col("count") > 1) \
      6     .select(emails_dataframe_grouped_1.email.alias("Email")) \
      7     .show()
      9 emails_dataframe_grouped_2 = emails_dataframe.groupBy(emails_dataframe.email).agg(
---> 10     count().alias("email_instance")
     11 )
     12 emails_dataframe_grouped_2 \
     13     .filter(emails_dataframe_grouped_2.email_instance > 1) \
     14     .select(emails_dataframe_grouped_1.email.alias("Email")) \
     15     .show()

NameError: name 'count' is not defined